In [104]:
import os
#---google cloud lib---------------
import json
import datetime
import gspread
import pandas as pd
from oauth2client.service_account import ServiceAccountCredentials
from pymongo import MongoClient
from google.cloud import bigquery
from bson import json_util, ObjectId
#---Google analyst lib-------------
from google.analytics.data_v1beta import BetaAnalyticsDataClient
from google.analytics.data_v1beta.types import DateRange
from google.analytics.data_v1beta.types import Dimension
from google.analytics.data_v1beta.types import Metric
from google.analytics.data_v1beta.types import RunReportRequest
from google.analytics.data_v1beta.types import Filter, FilterExpression

os.environ["GOOGLE_APPLICATION_CREDENTIALS"]="ga_token.json"

def sample_run_report(property_id, dim):
    """Runs a simple report on a Google Analytics 4 property."""
    # Using a default constructor instructs the client to use the credentials
    # specified in GOOGLE_APPLICATION_CREDENTIALS environment variable.
    client = BetaAnalyticsDataClient()
    request = RunReportRequest(
        property=f"properties/{property_id}",
        dimensions=[Dimension(name="customUser:mobile"), Dimension(name="audienceName")],
        metrics=[Metric(name="activeUsers")],
        date_ranges=[DateRange(start_date="yesterday", end_date="yesterday")],
        dimension_filter=FilterExpression(
                filter=Filter(
                    field_name="audienceName",
                    string_filter=Filter.StringFilter(value=dim, match_type = "CONTAINS"),
                )
        ),
        limit = 100000
    )
    response = client.run_report(request)
    return response

def push_data(dim):
    response = sample_run_report(215493450, dim)
    
    #get data response from GA
    tmp_list = []
    #     print("Report result:")
    for row in response.rows:
        tmp_list.append([row.dimension_values[0].value, row.dimension_values[1].value, row.metric_values[0].value])
        
    #modify dataframe
    df = pd.DataFrame(tmp_list)
    df.columns = ['mobile', 'age', 'ignore']
    
    #define schema
    schema = []
    new_col_name = df.columns
    for name in df.columns:
        schema.append(bigquery.SchemaField("_".join(name.split(" ")), bigquery.enums.SqlTypeNames.STRING))
    
    #convert to json
    result = df.to_json(orient="records")
    data = json.loads(result)
    
    #define config
    client = bigquery.Client.from_service_account_json('google_token.json')
    job_config = bigquery.LoadJobConfig(
        schema=schema,
        write_disposition="WRITE_TRUNCATE",
    )

    end = datetime.datetime.now().replace(hour=0, minute=0, second=0, microsecond=0) - datetime.timedelta(days=(1))
    date_string = end.strftime("%Y%m%d")
    
    #upload to bigquery
    table_id = 'taptap-c2c5f.manual_dwh.GA_' + dim +'_' + date_string

    print(table_id)
    #print(data)
    job = client.load_table_from_json(
        data, table_id, job_config=job_config
    ) 
    print(job.result())
    
list_dim = ['Interest_News', 'Age', 'Gender', 'Interest_Media_TV', 'Interest_ShopperValue', 'Interest_Sport'
            , 'Interest_Media_Music', 'Interest_Media', 'Interest_Tech_SocialMedia', 'Interest_Shoppers'
            , 'Interest_Tech_Technophiles', 'Interest_Tech_Technophiles']
for dim in list_dim:
    push_data(dim)

taptap-c2c5f.manual_dwh.GA_Age_20220428
LoadJob<project=taptap-c2c5f, location=US, id=76fd7453-6206-42f1-956e-e3ad138cc5fe>


In [1]:
import os
#---google cloud lib---------------
import json
import datetime
import gspread
import pandas as pd
from oauth2client.service_account import ServiceAccountCredentials
from pymongo import MongoClient
from google.cloud import bigquery
from bson import json_util, ObjectId
#---Google analyst lib-------------
from google.analytics.data_v1beta import BetaAnalyticsDataClient
from google.analytics.data_v1beta.types import DateRange
from google.analytics.data_v1beta.types import Dimension
from google.analytics.data_v1beta.types import Metric
from google.analytics.data_v1beta.types import RunReportRequest
from google.analytics.data_v1beta.types import Filter, FilterExpression

os.environ["GOOGLE_APPLICATION_CREDENTIALS"]="ga_token.json"

def sample_run_report(property_id, dim):
    """Runs a simple report on a Google Analytics 4 property."""
    # Using a default constructor instructs the client to use the credentials
    # specified in GOOGLE_APPLICATION_CREDENTIALS environment variable.
    client = BetaAnalyticsDataClient()
    request = RunReportRequest(
        property=f"properties/{property_id}",
        dimensions=[Dimension(name="customUser:mobile"), Dimension(name="audienceName")],
        metrics=[Metric(name="activeUsers")],
        date_ranges=[DateRange(start_date="yesterday", end_date="yesterday")],
        dimension_filter=FilterExpression(
                filter=Filter(
                    field_name="audienceName",
                    string_filter=Filter.StringFilter(value=dim, match_type = "CONTAINS"),
                )
        ),
        limit = 100000
    )
    response = client.run_report(request)
    return response

def push_data(dim):
    response = sample_run_report(215493450, dim)
    
    #get data response from GA
    tmp_list = []
    #     print("Report result:")
    for row in response.rows:
        tmp_list.append([row.dimension_values[0].value, row.dimension_values[1].value, row.metric_values[0].value])
        
    #modify dataframe
    df = pd.DataFrame(tmp_list)
    df.columns = ['mobile', 'age', 'ignore']
    
    #define schema
    schema = []
    new_col_name = df.columns
    for name in df.columns:
        schema.append(bigquery.SchemaField("_".join(name.split(" ")), bigquery.enums.SqlTypeNames.STRING))
    
    #convert to json
    result = df.to_json(orient="records")
    data = json.loads(result)
    
    #define config
    client = bigquery.Client.from_service_account_json('google_token.json')
    job_config = bigquery.LoadJobConfig(
        schema=schema,
        write_disposition="WRITE_TRUNCATE",
    )

    end = datetime.datetime.now().replace(hour=0, minute=0, second=0, microsecond=0) - datetime.timedelta(days=(1))
    date_string = end.strftime("%Y%m%d")
    
    #upload to bigquery
    table_id = 'taptap-c2c5f.manual_dwh.GA_' + dim +'_' + date_string

    print(table_id)
    #print(data)
    job = client.load_table_from_json(
        data, table_id, job_config=job_config
    ) 
    print(job.result())
    
list_dim = ['Churn']
for dim in list_dim:
    push_data(dim)

taptap-c2c5f.manual_dwh.GA_Churn_20220430
LoadJob<project=taptap-c2c5f, location=US, id=497d400d-50ac-4e08-ba9b-c87f8f9dd9d7>


In [12]:
import datetime
# import tzlocal

def tzlocal():
    from tzlocal import get_localzone
    tz = get_localzone()
    return tz

dt = {'Contents':
          [
                {'Key': 'order_mart/', 'ETag': '"d41d8cd98f00b204e9800998ecf8427e"', 'LastModified': datetime.datetime(2022, 4, 25, 4, 36, 55, tzinfo = tzlocal())},
                {'Key': 'order_mart/order_profile/', 'ETag': '"d41d8cd98f00b204e9800998ecf8427e"', 'LastModified': datetime.datetime(2022, 4, 25, 4, 36, 55, tzinfo = tzlocal())}
          ]
      }
value = dt['Contents']

print(value)
for obj in value:
    print(obj)
    print(obj['Key'])

[{'Key': 'order_mart/', 'ETag': '"d41d8cd98f00b204e9800998ecf8427e"', 'LastModified': datetime.datetime(2022, 4, 25, 4, 36, 55, tzinfo=_PytzShimTimezone(backports.zoneinfo.ZoneInfo(key='Asia/Bangkok'), 'Asia/Bangkok'))}, {'Key': 'order_mart/order_profile/', 'ETag': '"d41d8cd98f00b204e9800998ecf8427e"', 'LastModified': datetime.datetime(2022, 4, 25, 4, 36, 55, tzinfo=_PytzShimTimezone(backports.zoneinfo.ZoneInfo(key='Asia/Bangkok'), 'Asia/Bangkok'))}]
{'Key': 'order_mart/', 'ETag': '"d41d8cd98f00b204e9800998ecf8427e"', 'LastModified': datetime.datetime(2022, 4, 25, 4, 36, 55, tzinfo=_PytzShimTimezone(backports.zoneinfo.ZoneInfo(key='Asia/Bangkok'), 'Asia/Bangkok'))}
order_mart/
{'Key': 'order_mart/order_profile/', 'ETag': '"d41d8cd98f00b204e9800998ecf8427e"', 'LastModified': datetime.datetime(2022, 4, 25, 4, 36, 55, tzinfo=_PytzShimTimezone(backports.zoneinfo.ZoneInfo(key='Asia/Bangkok'), 'Asia/Bangkok'))}
order_mart/order_profile/
